<a href="https://colab.research.google.com/github/charlie9526/TaxiFare/blob/master/catboost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 . Mounting

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/FAIR_TAXI

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/FAIR_TAXI


# 2 . Imports

In [ ]:
import pandas as pd
from datetime import datetime
import random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)              
import numpy as np                 
from scipy.stats import norm   
import warnings
warnings.filterwarnings('ignore')   
%matplotlib inline                  
from math import sin, cos, sqrt, atan2, radians
import math

from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier

!pip install catboost
from catboost import CatBoostClassifier,Pool
import statistics


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 64.8MB 101kB/s 


# 3 . Read files

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 4 . Functions

In [ ]:
#elts write a lambda function
def get_duration(x):
  if x['pickup_time']==None or x['drop_time']==None:
    return None
  
  try :
    if int(x['pickup_time'].split("/")[1])>12 or int(x['drop_time'].split("/")[1])>12:
      gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
    else:
      if int(x['pickup_time'].split("/")[1]) != int(x['drop_time'].split("/")[1]):
        gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
      else:
        gap = datetime.strptime(x['pickup_time'],"%d/%m/%Y %H:%M")-datetime.strptime(x['drop_time'],"%d/%m/%Y %H:%M")
    # print (gap.total_seconds()/3600,x['pickup_time'] ,x['drop_time'])
  except :
    # print(int(x['pickup_time'].split("/")[0])>12 or int(x['drop_time'].split("/")[0])>12)
    return None
  # print(abs(gap.total_seconds()/3600), x['drop_time'],x['pickup_time'])
  disti =  abs(gap.total_seconds())
  return disti


# debugerr 
class debugerr(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        print (pd.DataFrame(X));
        return self
    def transform(self,X):
        return X

# lambda function for change label
def get_binary(x):
  if x==None:
    return 0
  if x == "correct":
    # print (x);
    return 1
  else:
    return 0

# set distance column
class set_distance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X['distance'] = X.apply(lambda x:get_distance(x), axis=1)
      # print(X)
      # X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X
# lambda column get distance
def get_distance(x):
  R = 6373.0
  lat1 = radians(float(x['drop_lat']))
  lon1 = radians(float(x['drop_lon']))
  lat2 = radians(float(x['pick_lat']))
  lon2 = radians(float(x['pick_lon']))

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

# remove columns
class remove_colums_unwanted(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X


def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

def get_hour_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  h =  datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").hour
  return h

def get_minute_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  h =  datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").minute
  return h

################### Here we have to add fare_per_hour column
################### write a lambda function

def get_fare_per_hr(x):
  if  x["duration"]>0:
    hours = x["duration"]
    val = float(x['fare'])/hours
    return val
  if x["duration"]==0:
    return 0
  return None

################### Here we have to add additional_fare_per_hour column
################### write a lambda function

def get_addfare_per_hr(x):
  if  x["distance"]>0:
    val = float(x['additional_fare'])/x["distance"]
    return val
  if x["distance"]==0:
    return 0
  return None

########### This is for getting fare per distance 
########### Lambda function
def get_dis_per_fare(x):
  if x['fare']==0:
    return 0
  if x['fare']>0:
    return float(x['distance'])/x['fare']
  else:
    return 0

################fare rate of waiting
def get_waiting_rate(x):
  if math.isnan(x['meter_waiting']):
    return 0
  if x['meter_waiting']==0:
    return 0
  if  x['meter_waiting_fare']>=0:
    return float(x['meter_waiting_fare'])/x['meter_waiting']
  else:
    return 0

#################distance per min
def get_velocity(x):
  if math.isnan(x['duration']) or math.isnan(x['distance']) or math.isnan(x['meter_waiting']):
    return 0
  if (x['duration']-x['meter_waiting'])==0:
    return 0
  return (float(x['distance']))/(x['duration']-x['meter_waiting'])


##################ratio of meter waiting from total duration
def get_mw_ratio(x):
  if (x['duration'])==0:
    return 0
  return float(x['meter_waiting'])/(x['duration'])

##################ratio of meter waiting fare from total fare
def get_mwfare_ratio(x):
  if (x['fare'])==0:
    return 0
  return float(x['meter_waiting_fare'])/(x['fare'])

################### Here we have to add fare_per_hour column
################### write a lambda function

def get_hour_per_fare(x):
  if  x["fare"]>0:
    fare = x["duration"]
    val = float(x['fare'])/fare
    return val
  if x["fare"]==0:
    return 0
  return None


# 5 . PreProcess

### Some pre steps


In [ ]:
train_df = train_df.drop_duplicates()

# train_df = train_df.dropna()

### Create feature_df and label

In [ ]:
trip_id  = test_df['tripid']

features_df = train_df.loc[:, train_df.columns != 'label']
label_df = train_df['label'].apply(encoding_label).values

features_df = features_df.loc[:, features_df.columns != 'tripid']
test_df = test_df.loc[:, test_df.columns != 'tripid']

### Fill nuls

In [ ]:


################### ugly indexes
bads = [203757140,213112837,193653017,209365576,190167541]
train_df = train_df[~(train_df['tripid'].isin(bads))]

# features_df['duration'] = features_df['duration']/float(3600)
# test_df['duration'] = test_df['duration']/float(3600)

# features_df['meter_waiting'] = features_df['meter_waiting']/float(3600)
# test_df['meter_waiting'] = test_df['meter_waiting']/float(3600)

# features_df['meter_waiting_till_pickup'] = features_df['meter_waiting_till_pickup']/float(3600)
# test_df['meter_waiting_till_pickup'] = test_df['meter_waiting_till_pickup']/float(3600)


### New columns

In [ ]:
features_df['distance'] = features_df.apply(lambda x:get_distance(x), axis=1)
test_df['distance'] = test_df.apply(lambda x:get_distance(x), axis=1)

features_df['fare_per_time'] = features_df['fare']/features_df['duration']
test_df['fare_per_time'] = test_df['fare']/test_df['duration']

features_df['run_time_per_fare'] = (features_df['duration']-features_df['meter_waiting'])/features_df['fare']
test_df['run_time_per_fare'] = (test_df['duration']-test_df['meter_waiting'])/test_df['fare']

features_df['run_time_per_run_fare'] = (features_df['duration']-features_df['meter_waiting'])/(features_df['fare']-features_df['meter_waiting_fare'])
test_df['run_time_per_run_fare'] = (test_df['duration']-test_df['meter_waiting'])/(test_df['fare']-test_df['meter_waiting_fare'])

features_df['add_fare_per_time'] = (features_df['additional_fare'])/(features_df['duration'])
test_df['add_fare_per_time'] = (test_df['additional_fare'])/(test_df['duration'])

features_df['meter_waiting_duration'] = (features_df['meter_waiting'])/(features_df['distance'])
test_df['meter_waiting_duration'] = (test_df['meter_waiting'])/(test_df['distance'])

features_df['pickup_hour'] = features_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)
test_df['pickup_hour'] = test_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)

features_df['pickup_min'] = features_df.apply(lambda x : get_minute_value(x,'pickup_time'),axis=1)
test_df['pickup_min'] = test_df.apply(lambda x : get_minute_value(x,'pickup_time'),axis=1)

features_df['drop_hour'] = features_df.apply(lambda x : get_hour_value(x,'drop_time'),axis=1)
test_df['drop_hour'] = test_df.apply(lambda x : get_hour_value(x,'drop_time'),axis=1)

features_df['drop_minute'] = features_df.apply(lambda x : get_minute_value(x,'drop_time'),axis=1)
test_df['drop_minute'] = test_df.apply(lambda x : get_minute_value(x,'drop_time'),axis=1)


features_df


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance,fare_per_time,run_time_per_fare,run_time_per_run_fare,add_fare_per_time,meter_waiting_duration,pickup_hour,pickup_min,drop_hour,drop_minute
0,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.094369,0.324125,2.878070,2.878070,0.012590,10.992530,0,20,0,34
1,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.169052,0.250126,3.760425,3.760425,0.013274,14.830933,0,56,1,9
2,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.307375,0.277498,3.338417,3.338417,0.009660,12.683566,1,8,1,26
3,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.862217,0.137625,3.973269,4.907243,0.017559,314.306135,2,27,2,37
4,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.150340,NaN,NaN,NaN,NaN,NaN,3,34,3,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.106037,0.236587,3.757692,3.863344,0.012530,44.158778,22,7,22,21
17172,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.871789,0.270214,2.964403,2.964403,0.004881,39.367945,23,7,23,43
17173,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045847,0.289734,3.333333,3.333333,0.039924,8.605468,23,21,23,25
17174,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879981,0.155373,5.573475,5.573475,0.012238,39.930824,23,39,23,53


### Drop Columns

In [ ]:
features_df = features_df.drop(labels=['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','meter_waiting_till_pickup'],axis=1)
test_df = test_df.drop(labels=['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','meter_waiting_till_pickup'],axis=1)

### Pipeline's pre prcessors

In [ ]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values
cat_cols = features_df.columns[features_df.dtypes == "object"].values

## **Catboost**


In [ ]:
x_train,x_test,y_train,y_test =  train_test_split( features_df, label_df, test_size=0.15, random_state=6)

data_pool = Pool(
    data=x_train,
    label=y_train
)
model = CatBoostClassifier(
  # feature_weights = "7:1.2,6:1.2",
  # iterations=5000,
  random_seed = 42,
  # depth = 7,
  eval_metric='F1'
) 
model.fit(
    data_pool,
    eval_set=(x_test, y_test),
    verbose = 800
)
eval_pool = Pool(
    data = x_test,
    label = y_test
)
eval_result = model.eval_metrics(
    data = eval_pool,
    metrics = ['F1']
)
for metric_key in eval_result:
  print ("%s -: %f" % (metric_key,100*statistics.mean(eval_result[metric_key])))
print(f1_score(y_test,model.predict(x_test,prediction_type='Class')))
predicted = model.predict(test_df,prediction_type='Class')



Learning rate set to 0.061443
0:	learn: 0.9699360	test: 0.9726753	best: 0.9726753 (0)	total: 15.1ms	remaining: 15.1s
800:	learn: 0.9917106	test: 0.9804422	best: 0.9812925 (173)	total: 10.1s	remaining: 2.52s
999:	learn: 0.9933600	test: 0.9795745	best: 0.9812925 (173)	total: 12.7s	remaining: 0us

bestTest = 0.981292517
bestIteration = 173

Shrink model to first 174 iterations.
[0.9726752806608769, 0.9735561667019252, 0.9754341380770859, 0.9736008447729673, 0.9731784582893348, 0.9742398648648649, 0.9758679085520745, 0.976281236764083, 0.9764780673871581, 0.9767146486028789, 0.9764880321965684, 0.9769116712560899, 0.9769116712560899, 0.9771089444679949, 0.977542372881356, 0.977542372881356, 0.977542372881356, 0.977316090735637, 0.9775328529037728, 0.977542372881356, 0.9773257045984318, 0.9773257045984318, 0.9773257045984318, 0.9771089444679949, 0.977316090735637, 0.9771089444679949, 0.977542372881356, 0.9771186440677966, 0.9771186440677966, 0.9771089444679949, 0.977316090735637, 0.97732570

In [ ]:
model = CatBoostClassifier(
  # feature_weights = "7:1.2,6:1.2",
  iterations=178,
  random_seed = 42,
  # depth = 7,
  eval_metric='F1'
) 
fit_pool = Pool(
    data=features_df,
    label=label_df
)
model.fit(
    fit_pool,
    verbose = 800
)
data_dict = { 'tripid':trip_id, 'prediction':predicted}
pd_test_result = pd.DataFrame(data=data_dict,columns=['tripid','prediction'])
pd_test_result = pd_test_result.set_index('tripid') 
pd_test_result.to_csv("catboost_result.csv", index=True)

Learning rate set to 0.168876
0:	learn: 0.9699571	total: 8.31ms	remaining: 1.47s
177:	learn: 0.9822870	total: 1.34s	remaining: 0us


F1 -: 0.976345 rs 42

# 9 . feature importnce

In [ ]:
pd_test_result['prediction'].value_counts()

1    8163
0     413
Name: prediction, dtype: int64